<a href="https://colab.research.google.com/github/IgorBaratta/FEniCSxCourse/blob/ICMC23/Problem4_Thermo-Elasticity/ThermoElastic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Thermo-Elastic deformation
---

In this tutorial we will combine two of our previous solvers to model the thermo-elastic deformation of a bimaterial, i.e., a specimen made of two materials with different thermal expansion coefficients. The idea is to solve a problem involving two fields governed by different EDPs, namely, the temperature field $T$ and the displacement field $\mathbf{u}$.

**Mathematical formulation:**

Let us first recall the stationary **heat conduction equation** that governs the thermal problem

\begin{equation}
\left \{
\begin{array}{rcll}
-\nabla \cdot ( \kappa \, {T}) & = & s &  \mbox{in}~\Omega \\
&& \\
{T} & = & T_D &  \mbox{in}~\partial\Omega_D \\
&& \\
-\kappa \nabla{T}\cdot\check{\mathbf{n}} & = & J_N &  \mbox{in}~\partial\Omega_N \\
\end{array}
\right.
\end{equation}

where $s(\mathbf{x})$ denotes the source term, $\kappa$ the thermal diffusivity,
$T_D$ the Dirichlet data and $J_N$ the Neumann data.

As for the **elastostatic** problem that governs the deformation of the specimen, we have

\begin{equation}
\left \{
\begin{array}{rcll}
-\nabla \cdot \boldsymbol{\sigma} (\mathbf{u},T) & = & \mathbf{f} & \mbox{in}~\Omega \\
& & & \\
\mathbf{u} & = & \mathbf{g} & \mbox{on}~\Gamma_{\mathbf{u}} \\
& & & \\
\boldsymbol{\sigma} \cdot \check{\mathbf{n}} & = & \boldsymbol{\mathcal{F}} & \mbox{on}~\Gamma_{\boldsymbol{\mathcal{F}}}
\end{array}
\right.
\end{equation}

where $\mathbf{f}$ is the body force, $\mathbf{g}$ the prescribed displacement, $\boldsymbol{\mathcal{F}}$ the surface force distribution.


The stress tensor is

$$
\boldsymbol{\sigma} = 2\mu\, \boldsymbol{\varepsilon}^{elas}({\mathbf{u}})+ \lambda \, \mbox{tr}(\boldsymbol{\varepsilon}^{elas}({\mathbf{u}}))\, \mathbf{I}_{d}
$$
   

  
where $d$ is the spatial dimension, $\mathbf{I}_{d}$ is the identity matrix and `tr` stands for the trace of a tensor, i.e.

$$
\mbox{tr}(\boldsymbol{\varepsilon}^{elas}({\mathbf{u}})) = 
\sum_{k=1}^d{\varepsilon}^{elas}_{kk}
$$

The difference now is that we have an additional contribution to the
strain tensor

$$
\boldsymbol{\varepsilon}({\mathbf{u}}) =
\frac12 (\nabla{\mathbf{u}} + \nabla^{\intercal}{\mathbf{u}}) = 
 \boldsymbol{\varepsilon}^{elas}({\mathbf{u}}) + \alpha(\mathbf{x})\, (T - T_0)\,\mathbf{I}_{d}
$$

where $T_0$ is a reference temperature at which no thermal deformation
exists, that we assume to be equal to $0$.

$~$

---

To implement the finite element solution, what matter to us is the **variational formulation of the coupled problem** given above, which is obtained by combining the two previously seen formulations for the Poisson's problem and the elasticity problem: Find $\mathbf{u} \in U_{\mathbf{g}}$ and $T \in V_{T}$ such that

\begin{eqnarray}
a_{elas}((\mathbf{u},T), \mathbf{v}) = \int_{\Omega}{\boldsymbol{\sigma}(\mathbf{u},T) : \boldsymbol{\varepsilon}(\mathbf{v}) \,dx} & = & \\
& = & \int_{\Omega}{\mathbf{f}\cdot \mathbf{v}}\,dx +
\int_{\Gamma_{\boldsymbol{\mathcal{F}}}}{\boldsymbol{\mathcal{F}} \cdot \mathbf{v}}\,ds = \ell_{elas}(\mathbf{v}) ~~~\forall \mathbf{v} \in U_{\mathbf{0}}\\
& & \\
a_{th}(T,r) = {\int_{\Omega}}{\kappa\,\nabla{T} \cdot \nabla{r}}\,dx & = & \\
& = &\int_{\Omega}{s\,r}\,dx - \int_{\partial\Omega_N}{J_N\,r}\,ds = \ell_{th}(r)~~~\forall r \in V_0
\end{eqnarray}

$~$

Of course, we can add everything together to obtain: Find $(\mathbf{u}, T) \in W = U_{\mathbf{g}} \times V_T$ such that 

$$
a\left ( (\mathbf{u}, T),  (\mathbf{v}, r) \right) = L((\mathbf{v}, r))
$$

$\forall (\mathbf{v}, r) \in U_{\mathbf{0}} \times V_0$ with

\begin{eqnarray}
a\left ( (\mathbf{u}, T),  (\mathbf{v}, r) \right) & = & 
a_{elas}((\mathbf{u},T), \mathbf{v}) + a_{th}(T,r)\\
&& \\
L((\mathbf{v}, r)) & = & \ell_{elas}(\mathbf{v}) + \ell_{th}(r)
\end{eqnarray}

The discrete problem follows by taking finite dimensional subspaces $U_h$ and
$V_h$, which leads to the following discrete system

$$
\begin{bmatrix}
A_{\mathbf{u}\mathbf{u}} & B_{\mathbf{u}T} \\
0 & C_{TT}
\end{bmatrix} 
\begin{bmatrix}
\mathsf{U} \\
\mathsf{T}
\end{bmatrix} 
= 
\begin{bmatrix}
\mathsf{F} \\
\mathsf{G}
\end{bmatrix} 
$$

where $\boldsymbol{\mathsf{X}} = [\mathsf{U}, \mathsf{T}]^{\intercal}$
is the global vector of displacement and temperature unknowns.

It is import to remark that, since the temperature field does not depend on the displacement $\mathbf{u}$, we could easily eliminate $T$ and substitute in the elasticity problem, i.e.,

$$
A_{\mathbf{u}\mathbf{u}} \mathsf{U} = \mathsf{F} - C_{TT} \mathsf{T}
$$

Nevertheless, for didactical reasons, in this tutorial we solve the problem
in monolithic form, so as to determine both fields at once and illustrate the use of **mixed function spaces**. Moreover, with this implementation, the extension to handle problems with a stronger coupling is expected to be easier.
A classical situation is when we include the influence of the volumetric changes into the energy balance in the transient case, i.e.,

$$
\rho C_p \dfrac{\partial{T}}{\partial{t}} - \nabla \cdot (\kappa \nabla{T}) + T_0 \, \alpha \,\mbox{tr}\left (\dfrac{\partial{{\boldsymbol{\varepsilon}}}}{\partial{t}} \right ) = s
$$

also, notice that the thermal conductivity $\kappa$ can depend on the current
level of stresses in the material.


## Initialization

As usual be begin by importing the necessary packages:

In [ ]:
try:
    import gmsh
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/gmsh-install.sh" -O "/tmp/gmsh-install.sh" && bash "/tmp/gmsh-install.sh"
    import gmsh
    
try:
    import dolfinx
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/fenicsx-install-real.sh" -O "/tmp/fenicsx-install.sh" && bash "/tmp/fenicsx-install.sh"
    import dolfinx

try:
    import pyvista
except ImportError:
    !pip install - q piglet pyvirtualdisplay ipyvtklink pyvista panel
    !apt-get - qq install xvfb
    import pyvista

In [ ]:
from dolfinx import mesh, fem, io, plot

from ufl import sin, SpatialCoordinate, FiniteElement, VectorElement, MixedElement, TestFunction, TrialFunction, split, Identity, Measure, dx, ds, grad, nabla_grad, div, dot, inner, tr, as_vector, FacetNormal

import numpy as np
from mpi4py import MPI
from petsc4py.PETSc import ScalarType
import matplotlib.pyplot as plt

## Geometry

The aim is to create a rectangular mesh of the computational domain $\Omega = [0,L_x] \times [0, L_y]$. The domain will be divided into two regions, namely, the bottom and the top part

$$
\Omega = \Omega_B \cup \Omega_T
$$

where $\Omega_B = [0,L_x] \times [0, L_y/2]$ and $\Omega_T = [0,L_x] \times [L_y/2, L_y]$ each having different material properties.

In [ ]:
Lx = 1.0
Ly = 0.05
widthlay = 0.5*Ly
lengthlay = Lx

def GenerateMesh():

    gmsh.initialize()
    proc = MPI.COMM_WORLD.rank

    if proc == 0:
        gmsh.model.occ.addRectangle(0, 0, 0, lengthlay, widthlay, tag=1)
        gmsh.model.occ.addRectangle(0, widthlay, 0, lengthlay, widthlay, tag=2)
        # We fuse the two rectangles and keep the interface between them 
        gmsh.model.occ.fragment([(2,1)],[(2,2)])
        gmsh.model.occ.synchronize()
   
        # Mark the top (2) and bottom (1) rectangle
        top, bottom = None, None
        for surface in gmsh.model.getEntities(dim=2):
          com = gmsh.model.occ.getCenterOfMass(surface[0], surface[1])
          if np.allclose(com, [0.5*Lx, 0.5*widthlay, 0]):
            bottom = surface[1]
          else:
            top = surface[1]

        gmsh.model.addPhysicalGroup(2, [bottom], 1)
        gmsh.model.addPhysicalGroup(2, [top], 2)

        # Tag the left and right boundaries
        left = []
        right = []
        for line in gmsh.model.getEntities(dim=1):
            com = gmsh.model.occ.getCenterOfMass(line[0], line[1])
            if np.isclose(com[0], 0.0):
                left.append(line[1])
            if np.isclose(com[0], Lx):
                right.append(line[1])
        gmsh.model.addPhysicalGroup(1, left, 3)
        gmsh.model.addPhysicalGroup(1, right,1)

        gmsh.model.mesh.setSize(gmsh.model.getEntities(0), 0.005)
        gmsh.model.mesh.generate(2)    
    
        msh, subdomains, boundaries = io.gmshio.model_to_mesh(gmsh.model, comm=MPI.COMM_WORLD, rank=0, gdim=2)
        
    gmsh.finalize()

    return msh, subdomains, boundaries

msh, subdomains, boundaries = GenerateMesh()

with io.XDMFFile(MPI.COMM_WORLD, "mymesh.xdmf", "w") as xdmf:
    xdmf.write_mesh(msh)

import IPython

def plot_mesh(mesh, cell_values=None, filename="file.html"):
    pyvista.start_xvfb()
    grid = pyvista.UnstructuredGrid(*plot.create_vtk_mesh(mesh))
    plotter = pyvista.Plotter(notebook=True, window_size=[500,500])

    if cell_values is not None:
        min_ = cell_values.x.array.min()
        max_ = cell_values.x.array.max()
        grid.cell_data["cell_values"] = cell_values.x.array
        viridis = plt.cm.get_cmap("viridis", 25)
        plotter.add_mesh(grid, cmap=viridis, show_edges=True, clim=[min_, max_])
    else:
        plotter.add_mesh(grid, show_edges=True)
    
    plotter.camera.zoom(2.0)
    plotter.view_xy()
    plotter.export_html(filename, backend="pythreejs")
    plotter.close()


plot_mesh(msh, cell_values = None, filename="mesh.html")
IPython.display.HTML(filename="mesh.html")

## Finite element spaces

The solution $(\mathbf{u}, T)$ belongs to the product space 
$W_h = U_h \times V_h$, $~~U_h \subset [H^1(\Omega)]^d$ and 
$V_h \subset H^1(\Omega)$. Since we are working with conforming 
finite element formulations, natural choices for the discrete spaces
$U_h$ and $V_h$ are:

$$
U_h = U(\mathcal{T}_h) = \{\mathbf{v} \in [H^1(\Omega)]^d,~\mathbf{v}|_E \in [P_k(E)]^d \, \forall E \in \mathcal{T}_h\}
$$

and 

$$
V_h = V(\mathcal{T}_h) = \{v \in H^1(\Omega),~v|_E \in P_k(E) \, \forall E \in \mathcal{T}_h\}
$$

In particular we will use polynomials of degree $1$, which is the most economical option for this case. In order to implement a monolithic formulation of the problem at hand, we need to create a mixed function space in `dolfinx` as follows 

In [ ]:
# Create an element for each field

degree_u = 1
el_u = VectorElement("CG", msh.ufl_cell(), degree_u)

degree_T = 1
el_T = FiniteElement("CG",  msh.ufl_cell(), degree_T)

# the mixed element 
el_m  = MixedElement([el_u , el_T])

# and the corresponding global space of functions
W = fem.FunctionSpace(msh, el_m)

# Finally, declare a space of piecewise constant functions for later use
Q = fem.FunctionSpace(msh, ("DG", 0))

## Boundary conditions

Setting up the boundary conditions take some extra work, since we
have to distinguish those BCs for the displacement field and for
the temperature field, namely,

* For the displacement field we totally restrict the left side

$$
\mathbf{u}(x=0) = [0,0]^{\intercal}
$$

* For the temperature field we impose:

$$
T(x=0) = T_l,~~~T(x=L_x) = T_r
$$

* The rest of the boundary is traction free and adiabatic, so we take

$$
\boldsymbol{\mathcal{F}} = [0,0]^{\intercal},~~~g_N = 0
$$

although other boundary conditions are certainly possible. 


In order to proceed with the implementation, recall that the essential BCs have to be provided as a list of `fem.dirichletbc` objects, but first, we have
to identify the associated degrees of freedom `dofs` to which the boundary
value must be apply:

In [ ]:
# Start an empty list for the Dirichlet BCs
bcs = []
sdim = msh.topology.dim

# Identify the facets
left_facets = boundaries.indices[boundaries.values == 3]
right_facets = boundaries.indices[boundaries.values == 1]

#---------------------------------
# BCs for the displacement field

U, _ = W.sub(0).collapse()
bc_dofs_ul = fem.locate_dofs_topological((W.sub(0), U), sdim-1, left_facets)

# Auxiliary function: Nao ha uma forma mais facil!!!
def g(x):
  return (np.zeros_like(x[0]), np.zeros_like(x[0]))
w_bc = fem.Function(U)
w_bc.interpolate(lambda x: g(x))
bcs.append(fem.dirichletbc(w_bc, bc_dofs_ul, W.sub(0)))

#---------------------------------
# BCs for the temperature field

V, _ = W.sub(1).collapse()

bc_dofs_Tl = fem.locate_dofs_topological((W.sub(1), V), sdim-1, [left_facets])
bc_dofs_Tr = fem.locate_dofs_topological((W.sub(1), V), sdim-1, [right_facets])

# Boundary values
Tleft  = 0.0
Tright = 10.0

# Auxiliary function
def g(x, Temp):
  return (Temp*np.ones_like(x[0]))

w_bcl = fem.Function(V)
w_bcl.interpolate(lambda x: g(x, Tleft))
bcs.append(fem.dirichletbc(w_bcl, bc_dofs_Tl, W.sub(1)))

w_bcr = fem.Function(V)
w_bcr.interpolate(lambda x: g(x, Tright))
bcs.append(fem.dirichletbc(w_bcr, bc_dofs_Tr, W.sub(1)))


## Material parameters

Finally, let us recall the different terms in the variational formulation


For simplicity we will assume both materials have the same elastic parameters
that we take as, in terms of the Young's modulus and Poisson ration

$$
E = 10,~~~~\nu = 0.3, 
$$


although they have different thermal expansion coefficients, so as to illustrate the behavior of a bimetal bar, i.e.,

$$
\alpha(\mathbf{x}) = \left \{
\begin{array}{rcl}
\alpha_B = 10^{-3} & \mbox{if} & y < \frac12 L_y \\
\alpha_T = 10^{-5} & \mbox{if} & y >= \frac12 L_y
\end{array}
\right.
$$

and a thermal conductivity

$$
\kappa = 1
$$


In [ ]:
# Elastic constitutive parameters
E, nu = 10.0, 0.3
mu    = E/(2.0*(1.0 + nu))
lamb  = E*nu/((1.0 + nu)*(1.0 - 2.0*nu))

# Thermal conductivity
kappa = fem.Constant(msh, ScalarType(1.0))

# Thermal expansion
def SetAlpha(msh, subdomains, alphaB, alphaT):
  
  alpha = fem.Function(Q)

  # Identify elements with different tags
  cells_layerB = subdomains.indices[subdomains.values == 1]
  cells_layerT = subdomains.indices[subdomains.values == 2]
  
  alpha.x.array[cells_layerB] = np.full(len(cells_layerB), alphaB)
  alpha.x.array[cells_layerT] = np.full(len(cells_layerT), alphaT)
  
  return alpha

alphah = SetAlpha(msh, subdomains, 1e-3, 1e-5)

# Visualize in Paraview for verification
alphah.name = "alpha"
with io.XDMFFile(msh.comm, "alpha.xdmf", "w") as xdmf:
    xdmf.write_mesh(msh)
    xdmf.write_function(alphah)

plot_mesh(msh, cell_values=alphah, filename="alphavalues.html")
IPython.display.HTML(filename="alphavalues.html")

As for the body forces, we will take

$$
\mathbf{f} = [0, 0]^{\intercal}
$$

and heat source

$$
s = 100
$$

always assuming consistent units.

In [ ]:
# Body force
f = fem.Constant(msh, ScalarType( (0.0, 0.0) ) )

# Heat source
s = fem.Constant(msh, ScalarType(100.0))

## Variational formulation

In [ ]:
# Strain measures
def epsilon(u):
    return 0.5*(nabla_grad(u) + nabla_grad(u).T)

def epsilon_elas(u, T):
  return epsilon(u) - alphah*T*Identity(sdim)

def sigma(u, T):
    return 2*mu*epsilon_elas(u, T) + lamb*tr(epsilon_elas(u, T))*Identity(sdim)

TrialF = TrialFunction(W)
TestF = TestFunction(W)
(u, T) = split(TrialF)
(v, r) = split(TestF)

# LHS: Bilinear forms
a  = inner(sigma(u,T), epsilon(v)) * dx
a += inner(kappa*grad(T), grad(r)) * dx

# RHS: Forcing terms
L = inner(f, v)*dx
L += s*r*dx


Finally solve and save the solution for visualization in `Paraview`

In [ ]:
# Solver options
petsc_opts={"ksp_type": "preonly", "pc_type": "lu", "pc_factor_mat_solver_type": "mumps", "ksp_monitor": None}
problem = fem.petsc.LinearProblem(a, L, bcs=bcs, petsc_options=petsc_opts)
wh = problem.solve()

(uh,Th) = wh.split()

uh.name = "Displacement"
with io.XDMFFile(msh.comm, "disp.xdmf", "w") as xdmf:
    xdmf.write_mesh(msh)
    xdmf.write_function(uh)

Th.name = "Temperature"
with io.XDMFFile(msh.comm, "temp.xdmf", "w") as xdmf:
    xdmf.write_mesh(msh)
    xdmf.write_function(Th)

from google.colab import files
files.download('disp.xdmf')
files.download('disp.h5')
files.download('temp.xdmf')
files.download('temp.h5')

## Homework 5

1) Solve the case of a source term in the thermal problem given by
the function

$$
s(\mathbf{x}) = 100 \, \sin(2\pi\,x)
$$

which can be programmed as:

    def Source(x):
      return 100*sin(2*np.pi*x[0])

    x = SpatialCoordinate(msh)
    s = Source(x)

Take for simplicity the same boundary conditions of the previous example.


2) Consider the case in which $T_l = T_r = 10$ and $s = 0$, such that the temperature increment in the sample is simply 
$\Delta{T} = 10$ (i.e., uniform temperature distribution). 

Program a loop over different values of $\alpha_A$ to solve the thermoelastic problem and plot the maximum deflection
 of the bimetal beam as a function of $\alpha_A$.
 Notice that in the previous implementation we regarded
 many parameters, such as `alphah` as having global scope,
 so, you should structure the code as follows in order 
 to be able to update $\alpha_A$:

        def epsilon_elas(u, T, alpha):
          return epsilon(u) ...

        def sigma(u, T, alpha):
          return ...

        max_def = []
        alphas = [1e-5, 1e-4, 1e-3]
        for alphaA in alphas:

          print('Solving for alphaA=', alphaA)
          alphah = SetAlpha(msh, subdomains, alphaA, 1e-5)

          a  = ...
          a += inner(kappa*grad(T), grad(r)) * dx
          .
          .
          .
          problem = fem.petsc.LinearProblem(a, L, bcs=bcs, petsc_options=petsc_opts)
          wh = problem.solve()
          (uh,Th) = ...

          max_def.append( ... )
          uh.name = "Displacement"
          xdmf_f.write_function(uh, alphaA)



Compare to the analytical prediction based on beam theory

$$
\delta_{\max} = \dfrac{12(\alpha_B - \alpha_T) \Delta{T}\, L_x^2}{L_y \,K}
$$

where 

$$
K = 14 + \dfrac{E_B}{E_T} + \dfrac{E_T}{E_B}
$$

where $E_B$ and $E_T$ are the Young's modulus of the
bottom and the top layer of the bimetal. Since, we are 
assuming the same values for both, we have $K = 16$.